# Import Libraries

In [ ]:
from sched import scheduler
!pip install gpytorch torchsummary iterative-stratification optuna pytorch_metric_learning wandb
!pip install torch pydicom pandas scikit-learn scikit-image numpy opencv-python matplotlib

In [ ]:
import optuna
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import wandb
from datetime import datetime

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms
from pytorch_metric_learning import losses

from sklearn.model_selection import train_test_split
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score
from sklearn.metrics import roc_curve, auc, confusion_matrix, ConfusionMatrixDisplay, roc_auc_score, \
    multilabel_confusion_matrix
from sklearn.model_selection import KFold, StratifiedKFold

import sys

sys.path.append('../')

import models.mil_resnet as MILModels
from utils import hard_negative_mining as hnm
import gpytorch
from layers.gaussian_process import SingletaskGPModel, PGLikelihood
from utils.early_stopping import EarlyStoppingForOptimization, EarlyStopping

os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
import gc

torch.cuda.empty_cache()
gc.collect()

In [3]:
import warnings
from sklearn.exceptions import UndefinedMetricWarning

warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)

# Configurations
## GPU Configurations

In [ ]:
# Initialize GPU Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)} is available.")
else:
    print("No GPU available. Training will run on CPU.")

print(device)

In [5]:
%load_ext autoreload
%autoreload 2

## Seed Everything

In [6]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything()

## Constants and Hyperparameters

In [7]:
import yaml

with open("../config.yaml") as file:
    config = yaml.safe_load(file)

# Accessing constants from config
HEIGHT = config['height']
WIDTH = config['width']
CHANNELS = config['channels']

TRAIN_BATCH_SIZE = config['train_batch_size']
VALID_BATCH_SIZE = config['valid_batch_size']
TEST_BATCH_SIZE = config['test_batch_size']
TEST_SIZE = config['test_size']
VALID_SIZE = config['valid_size']

TRAINING_TYPE = config['training_type']
DATA_REDUNDANCY = config['data_redundancy']
GP_MODEL = config['gp_model']
GP_KERNEL = config['kernel_type']
MODEL_TYPE = config['model_type']
CONTRASTIVE_LEARNING = config['contrastive_learning']

MAX_SLICES = config['max_slices']
SHAPE = tuple(config['shape'])

NUM_EPOCHS = config['num_epochs']
LEARNING_RATE = config['learning_rate']
LEARNING_RATE_NGD = config['learning_rate_ngd']
INDUCING_POINTS = config['inducing_points']
THRESHOLD = config['threshold']

NUM_CLASSES = config['num_classes']

TARGET_LABELS = config['target_labels']

MODEL_PATH = config['model_path']
LIKELIHOOD_PATH = config['likelihood_path']
DEVICE = config['device']

PROJECTION_LOCATION = config['projection_location']
PROJECTION_HIDDEN_DIM = config['projection_hidden_dim']
PROJECTION_OUTPUT_DIM = config['projection_output_dim']

ATTENTION_HIDDEN_DIM = config['attention_hidden_dim']

In [ ]:
KAGGLE = os.path.exists(('kaggle/input'))
REMOTE_SERVER = os.path.exists(('/workspace/rsna-ich-mil'))
ROOT_DIR = None

if KAGGLE:
    DATA_DIR = ROOT_DIR + 'rsna-mil-training/'
    DICOM_DIR = DATA_DIR
    CSV_PATH = DICOM_DIR + 'training_1000_scan_subset.csv'
elif REMOTE_SERVER:
    DATA_DIR = '/root/rsna-ich-mil/'
    DICOM_DIR = DATA_DIR
    if DATA_REDUNDANCY:
        CSV_PATH = DATA_DIR + 'training_dataset_1150_redundancy.csv'
    else:
        CSV_PATH = '/workspace/training_dataset_1150.csv'
    print('Running on remote server.')
else:
    DATA_DIR = '../rsna-ich-mil/'
    DICOM_DIR = DATA_DIR
    if DATA_REDUNDANCY:
        CSV_PATH = './data_analyze/training_dataset_1150_redundancy.csv'
    else:
        CSV_PATH = './data_analyze/training_dataset_1150.csv'
    print(f'CSV Path: {CSV_PATH}')

patient_scan_labels = pd.read_csv(CSV_PATH, nrows=1150)
# patient_scan_labels = pd.read_csv(CSV_PATH)
dicom_dir = DICOM_DIR if KAGGLE else DATA_DIR

In [ ]:
patient_scan_labels.head()

# Data Preprocessing
## Splitting Data

In [10]:

def split_dataset(patient_scan_labels, test_size=TEST_SIZE, val_size=VALID_SIZE, random_state=42):
    # Extract the labels from the DataFrame
    labels = patient_scan_labels['patient_label']
    if test_size > 0:
        # First, split off the test set
        train_val_labels, test_labels = train_test_split(
            patient_scan_labels,
            test_size=test_size,
            stratify=labels,
            random_state=random_state
        )
        # Calculate the validation size relative to the train_val set
        val_size_adjusted = val_size / (1 - test_size)

        # Split the train_val set into train and validation sets
        train_labels, val_labels = train_test_split(
            train_val_labels,
            test_size=val_size_adjusted,
            stratify=train_val_labels['patient_label'],
            random_state=random_state
        )
    else:
        train_labels, val_labels = train_test_split(
            patient_scan_labels,
            test_size=val_size,
            stratify=labels,
            random_state=random_state
        )
        test_labels = None

    return train_labels, val_labels, test_labels

def split_dataset_for_multilabel(patient_scan_labels, test_size=0.15, val_size=VALID_SIZE, random_state=42):
    # Extract the labels from the DataFrame
    labels = patient_scan_labels[['patient_any', 'patient_epidural', 'patient_intraparenchymal',
                                  'patient_intraventricular', 'patient_subarachnoid', 'patient_subdural']].values

    if test_size > 0:
        # First split: train + test
        msss = MultilabelStratifiedShuffleSplit(n_splits=1, test_size=test_size, random_state=random_state)
        train_idx, test_idx = next(msss.split(patient_scan_labels, labels))

        train_labels = patient_scan_labels.iloc[train_idx]
        test_labels = patient_scan_labels.iloc[test_idx]

        # Second split: train + validation
        msss_val = MultilabelStratifiedShuffleSplit(n_splits=1, test_size=val_size, random_state=random_state)
        train_idx, val_idx = next(msss_val.split(train_labels, labels[train_idx]))

        train_labels_final = train_labels.iloc[train_idx]
        val_labels = train_labels.iloc[val_idx]

    else:
        # Only split into train and validation if test_size is 0
        msss_val = MultilabelStratifiedShuffleSplit(n_splits=1, test_size=val_size, random_state=random_state)
        train_idx, val_idx = next(msss_val.split(patient_scan_labels, labels))

        train_labels_final = patient_scan_labels.iloc[train_idx]
        val_labels = patient_scan_labels.iloc[val_idx]
        test_labels = None

    return train_labels_final, val_labels, test_labels

## Dataset Augmentor

In [11]:
class DatasetAugmentor:
    def __init__(self, height, width, levels=2, seed=None):
        self.height = height
        self.width = width
        self.levels = levels  # Dynamic number of levels
        self.seed = seed
        self.params = []

        # Create different levels of transforms based on the number of levels specified
        for i in range(levels):
            factor = (i + 1) / levels
            self.params.append(
                self._create_transform(
                    degrees=int(15 * factor),
                    translate_range=(0.2 * factor, 0.2 * factor),
                    scale_range=(1 - 0.2 * factor, 1 + 0.2 * factor),
                    brightness_range=0.2 * factor,
                    contrast_range=0.2 * factor,
                    blur_sigma_range=(0.5 * factor, 1.0 * factor),
                    apply_elastic=(i >= levels // 2),
                    level_name=f'level_{i + 1}'
                )
            )

    def _sample_value(self, value_range):
        if isinstance(value_range, tuple):
            random.seed(self.seed)
            return random.uniform(value_range[0], value_range[1])
        return value_range

    def _create_transform(self, degrees, translate_range, scale_range, brightness_range, contrast_range,
                          blur_sigma_range, apply_elastic, level_name):
        print(f"Creating '{level_name}' transform with parameters:")
        sampled_values = {
            "degrees": abs(self._sample_value((-degrees, degrees))),
            "translate": (abs(self._sample_value(translate_range[0])), abs(self._sample_value(translate_range[1]))),
            "scale": self._sample_value(scale_range),
            "brightness": self._sample_value(brightness_range),
            "contrast": self._sample_value(contrast_range),
            "blur_sigma": self._sample_value(blur_sigma_range),
            "apply_elastic": apply_elastic
        }

        print(sampled_values)
        return sampled_values

    def apply_transform(self, image, level):
        params = self.params[level]
        transform = self._get_transform(params, channels=image.shape[0])
        return transform(image)

    def _get_transform(self, params, channels=3):
        transform_list = [
            transforms.ToPILImage(),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomAffine(degrees=params["degrees"], translate=params["translate"],
                                    scale=(params["scale"], params["scale"])),
            transforms.ColorJitter(brightness=params["brightness"], contrast=params["contrast"]),
            transforms.GaussianBlur(kernel_size=(3, 3), sigma=params["blur_sigma"]),
            transforms.RandomApply([transforms.ElasticTransform()] if params["apply_elastic"] else [], p=0.3),
            transforms.Resize(256),
            transforms.CenterCrop(self.height),
            # transforms.Compose([transforms.ToImage(), transforms.ToDtype(torch.float32, scale=True)])
            transforms.ToTensor()
        ]

        return transforms.Compose(transform_list)

    def _channel_shuffle(self, tensor):
        torch.manual_seed(self.seed)
        channels = tensor.shape[0]
        indices = torch.randperm(channels)
        return tensor[indices]

## Dataset Generator

In [12]:
from dataset_generators.RSNA_Dataset import MedicalScanDataset

In [13]:
class TrainDatasetGenerator(MedicalScanDataset):
    """Dataset class for training medical scan data."""
    def __init__(self, data_dir, patient_scan_labels, augmentor=None):
        super().__init__(data_dir, patient_scan_labels, augmentor)


class TestDatasetGenerator(MedicalScanDataset):
    """Dataset class for testing medical scan data."""
    def __init__(self, data_dir, patient_scan_labels, augmentor=None):
        super().__init__(data_dir, patient_scan_labels, augmentor)

In [ ]:
augmentor = DatasetAugmentor(HEIGHT, WIDTH, levels=2, seed=42)

In [ ]:
original_dataset = TrainDatasetGenerator(dicom_dir, patient_scan_labels, augmentor=None)

In [ ]:
len(original_dataset)

In [ ]:
x, y, z, _ = original_dataset[0]
print(x.shape, y.shape, z.shape)

In [18]:
def get_train_loader(dicom_dir, patient_scan_labels, batch_size=TRAIN_BATCH_SIZE):
    original_dataset = TrainDatasetGenerator(dicom_dir, patient_scan_labels, augmentor=None)
    return DataLoader(original_dataset, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True,
                      drop_last=True)

def get_test_loader(dicom_dir, patient_scan_labels, batch_size=TEST_BATCH_SIZE):
    test_dataset = TestDatasetGenerator(dicom_dir, patient_scan_labels, augmentor=None)
    return DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=8, pin_memory=True)

# Utils
## Augment batch for CL

In [19]:
# Version 2: Avg time taken: 0.05 seconds for 1 augmentation (w ResizedCrop)
def augment_batch(batch_images):
    if CHANNELS == 1:
        batch_size, num_instances, channels, height, width = batch_images.shape
    else:
        batch_size, num_instances, height, width, channels = batch_images.shape

    # Define augmentation transformations using GPU-compatible operations
    aug_transform = transforms.Compose([
        transforms.RandomResizedCrop((height, width), scale=(0.8, 1.1)),
        transforms.RandomApply([transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.4)], p=0.6),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.Compose([transforms.ToImage(), transforms.ToDtype(torch.float32, scale=True)]),
    ])

    # Apply transformations directly on the tensor without converting to PIL
    augmented_batch = torch.empty_like(batch_images)  # Preallocate memory for augmented images

    for i in range(batch_size):
        for j in range(num_instances):
            if CHANNELS == 1:
                augmented_batch[i, j] = aug_transform(batch_images[i, j])
            else:
                augmented_batch[i, j] = aug_transform(batch_images[i, j].permute(2, 0, 1)).permute(1, 2, 0)

    return augmented_batch.cuda()  # Move the augmented batch to GPU

## NTXentLoss

In [20]:
class NTXentLoss(losses.NTXentLoss):
    def __init__(self, temperature, **kwargs):
        super().__init__(temperature=temperature, **kwargs)
        self.temperature = temperature

    def forward(self, embeddings, labels=None, hard_pairs=None):
        # Normalize feature vectors
        feature_vectors_normalized = F.normalize(embeddings, p=2, dim=1)

        if labels == None:
            # Self-supervised labels
            batch_size = feature_vectors_normalized.size(0) // 2  # Assuming equal size for both embeddings
            labels = torch.cat([torch.arange(batch_size), torch.arange(batch_size)], dim=0)

        # Compute logits
        logits = torch.div(
            torch.matmul(
                feature_vectors_normalized, torch.transpose(feature_vectors_normalized, 0, 1)
            ),
            self.temperature,
        )

        if labels == None:
            return losses.NTXentLoss(temperature=self.temperature)(logits, torch.squeeze(labels))
        if hard_pairs == None:
            return losses.SupConLoss(temperature=self.temperature)(logits, torch.squeeze(labels))
        return losses.SupConLoss(temperature=self.temperature)(logits, torch.squeeze(labels), hard_pairs)

# Training and Validation
## Metrics Calculation
### Performance Metrics

In [21]:
def calculate_metrics(predictions, labels, average='weighted'):
    if NUM_CLASSES == 1:
        return {
            "accuracy": accuracy_score(labels, predictions),
            "precision": precision_score(labels, predictions, average=average),
            "recall": recall_score(labels, predictions, average=average),
            "f1": f1_score(labels, predictions, average=average),
            "auc": roc_auc_score(labels, predictions),
            "cohen_kappa": cohen_kappa_score(labels, predictions)
        }
    else:
        return {
            "accuracy": accuracy_score(labels, predictions),
            "precision": precision_score(labels, predictions, average='samples'),
            "recall": recall_score(labels, predictions, average='samples'),
            "f1": f1_score(labels, predictions, average='samples'),
            "auc": roc_auc_score(labels, predictions),
            "cohen_kappa": cohen_kappa_score(labels, predictions)
        }

def print_epoch_stats(epoch, num_epochs, phase, loss, metrics):
    """Print statistics for an epoch."""
    print(f"Epoch {epoch + 1}/{num_epochs} - {phase.capitalize()}:")
    print(f"Loss: {loss:.4f}, Accuracy: {metrics['accuracy']:.4f}, "
          f"Precision: {metrics['precision']:.4f}, Recall: {metrics['recall']:.4f}, "
          f"F1: {metrics['f1']:.4f}, AUC: {metrics['auc']:.4f}")

## Training Loop

### Training Phase 1: CNN + ATT

In [22]:
def train_phase_1(model, data_loader, criterion_cl, criterion_bce, optimizer, scheduler, device):
    total_loss = 0.0
    alpha = 0.5
    predictions = []
    labels = []

    model.train()

    for batch_data, batch_labels, batch_patient_labels, batch_multi_labels in tqdm(data_loader):
        batch_data = batch_data.to(device)
        batch_patient_labels = batch_patient_labels.float().to(device)
        batch_multi_labels = batch_multi_labels.float().to(device)
        optimizer.zero_grad()

        train_labels = batch_patient_labels if NUM_CLASSES == 1 else batch_multi_labels

        if CONTRASTIVE_LEARNING:
            aug_data_1 = augment_batch(batch_data)
            aug_data_2 = augment_batch(batch_data)
            batch_data = torch.cat([aug_data_1, aug_data_2], dim=0)
            train_labels = torch.cat([train_labels, train_labels], dim=0)

            outputs, cnn_features, attention_weights = model(batch_data)
            miner_func = hnm.ExamplePairMiner()
            hard_pairs = miner_func(outputs, train_labels)
            loss = criterion_cl(outputs, labels=train_labels, hard_pairs=hard_pairs) * alpha + \
                     criterion_bce(outputs, train_labels) * (1 - alpha)
            loss = loss.mean()
            preds = (torch.sigmoid(outputs) >= THRESHOLD).int()

        else:
            outputs, cnn_features, attention_weights = model(batch_data)
            loss = criterion_bce(outputs, train_labels)
            loss = loss.mean()
            preds = (torch.sigmoid(outputs) >= THRESHOLD).int()

        predictions.extend(preds.cpu().detach().numpy())
        labels.extend(train_labels.cpu().numpy())

        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        scheduler.step()

    return total_loss / len(data_loader), predictions, labels

def validate_phase_1(model, data_loader, criterion_cl, criterion_bce, device):
    """Validate the model."""
    model.eval()

    total_loss = 0.0
    alpha = 0.5
    predictions = []
    labels = []

    with (torch.inference_mode()):
        for batch_data, batch_labels, batch_patient_labels, batch_multi_labels in tqdm(data_loader):
            batch_data = batch_data.to(device)
            batch_patient_labels = batch_patient_labels.float().to(device)
            batch_multi_labels = batch_multi_labels.float().to(device)

            val_labels = batch_patient_labels if NUM_CLASSES == 1 else batch_multi_labels

            if CONTRASTIVE_LEARNING:
                outputs, cnn_features, attention_weights = model(batch_data)
                miner_func = hnm.ExamplePairMiner()
                hard_pairs = miner_func(outputs, val_labels)
                loss = criterion_cl(outputs, labels=val_labels, hard_pairs=hard_pairs) * alpha + \
                         criterion_bce(outputs, val_labels) * (1 - alpha)
                loss = loss.mean()
                preds = (torch.sigmoid(outputs) >= THRESHOLD).int()

            else:
                outputs, cnn_features, attention_weights = model(batch_data)
                loss = criterion_bce(outputs, val_labels)
                loss = loss.mean()
                preds = (torch.sigmoid(outputs) >= THRESHOLD).int()

            predictions.extend(preds.cpu().detach().numpy())
            labels.extend(val_labels.cpu().numpy())

            total_loss += loss.item()

    return total_loss / len(data_loader), predictions, labels

def evaluate_phase_1(model, data_loader, criterion_cl, criterion_bce, device):
    """Evaluate the model."""
    model.to(device)
    model.eval()

    total_loss = 0.0
    predictions = []
    probabilities = []
    labels = []

    with (torch.inference_mode()):
        for batch_data, batch_labels, batch_patient_labels, batch_multi_labels in tqdm(data_loader):
            batch_data = batch_data.to(device)
            batch_patient_labels = batch_patient_labels.float().to(device)
            batch_multi_labels = batch_multi_labels.float().to(device)

            eval_labels = batch_patient_labels if NUM_CLASSES == 1 else batch_multi_labels

            if CONTRASTIVE_LEARNING:
                outputs, cnn_features, attention_weights = model(batch_data)
                loss = criterion_cl(outputs, labels=eval_labels) * 0.5 + criterion_bce(outputs, eval_labels) * 0.5
                loss = loss.mean()
                preds = (torch.sigmoid(outputs) >= THRESHOLD).int()

            else:
                outputs, cnn_features, attention_weights = model(batch_data)
                loss = criterion_bce(outputs, eval_labels)
                loss = loss.mean()
                probs = torch.sigmoid(outputs)
                preds = (probs >= THRESHOLD).int()

            probabilities.extend(probs.cpu().detach().numpy())
            predictions.extend(preds.cpu().detach().numpy())
            labels.extend(eval_labels.cpu().numpy())

            total_loss += loss.item()

    return predictions, labels, probabilities

def train_model_phase_1(model, train_loader, val_loader, criterion_cl, criterion_bce, optimizer, num_epochs, device):
    best_model_wts = model.state_dict()
    best_auc = 0.0
    best_metrics = None
    model.to(device)

    scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=LEARNING_RATE, steps_per_epoch=len(train_loader), epochs=num_epochs)

    for epoch in range(num_epochs):
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
                loss, predictions, labels = train_phase_1(model, train_loader, criterion_cl, criterion_bce,
                                                           optimizer, scheduler, device)
            else:
                model.eval()
                loss, predictions, labels = validate_phase_1(model, val_loader, criterion_cl, criterion_bce, device)

            metrics = calculate_metrics(predictions, labels)
            print_epoch_stats(epoch, num_epochs, phase, loss, metrics)

            if phase == 'val' and best_auc < metrics['auc']:
                best_auc = metrics['auc']
                best_metrics = metrics
                best_model_wts = model.state_dict()

    model.load_state_dict(best_model_wts)
    return model, best_metrics

### Training Phase 2: GP

In [23]:
def train_epoch(model, likelihoods, data_loader, criterion_cl, criterion_bce, mlls, optimizer,
                variational_ngd_optimizer, scheduler, device):
    total_loss = 0.0
    alpha = 0.5
    predictions = []
    labels = []

    model.train()
    likelihoods.train()

    for batch_data, batch_labels, batch_patient_labels, batch_multi_labels in tqdm(data_loader):
        batch_data = batch_data.to(device)
        batch_patient_labels = batch_patient_labels.float().to(device)
        batch_multi_labels = batch_multi_labels.float().to(device)
        optimizer.zero_grad()

        if GP_MODEL == 'single_task':
            if NUM_CLASSES != 1:
                for i in range(NUM_CLASSES):
                    variational_ngd_optimizer[i].zero_grad()
            else:
                variational_ngd_optimizer.zero_grad()

        if TRAINING_TYPE == 'end_to_end':
            if CONTRASTIVE_LEARNING:
                aug_data_1 = augment_batch(batch_data)
                aug_data_2 = augment_batch(batch_data)
                batch_data = torch.cat([aug_data_1, aug_data_2], dim=0)
                batch_labels = torch.cat([batch_labels, batch_labels], dim=0)
                batch_multi_labels = torch.cat([batch_multi_labels, batch_multi_labels], dim=0)
                batch_patient_labels = torch.cat([batch_patient_labels, batch_patient_labels], dim=0)

            outputs, gp_outputs, att_outputs = model(batch_data)

            if GP_MODEL == 'single_task' and NUM_CLASSES != 1:
                loss = 0
                gp_loss = 0
                ntx_loss = 0

                for i in range(NUM_CLASSES):
                    gp_loss += -mlls[i](gp_outputs[i], batch_multi_labels[:, i])

                if CONTRASTIVE_LEARNING:
                    miner_func = hnm.ExamplePairMiner()
                    hard_pairs = miner_func(outputs, batch_multi_labels)
                    ntx_loss = criterion_cl(outputs, labels=batch_multi_labels, hard_pairs=hard_pairs)
                    loss = 0.3 * criterion_bce(outputs, batch_multi_labels) + 0.3 * gp_loss + ntx_loss * 0.4
                else:
                    loss = 0.5 * criterion_bce(outputs, batch_multi_labels) + 0.5 * gp_loss

                loss = loss.mean()

                # probs = [likelihoods[i](gp_outputs[i]).probs for i in range(NUM_CLASSES)]
                probs = [torch.sigmoid(outputs[:, i]) for i in range(NUM_CLASSES)]
                probabilities = torch.stack(probs, dim=1)
                preds = (probabilities >= THRESHOLD).int()

            elif GP_MODEL == 'single_task' and NUM_CLASSES == 1:
                if CONTRASTIVE_LEARNING:
                    miner_func = hnm.ExamplePairMiner()
                    hard_pairs = miner_func(outputs, batch_patient_labels)
                    ntx_loss = criterion_cl(outputs, labels=batch_patient_labels, hard_pairs=hard_pairs)
                    # loss = 0.3 * criterion_bce(outputs.squeeze(-1), batch_patient_labels) + 0.3 * mlls(gp_outputs, batch_patient_labels) + ntx_loss * 0.4
                    loss = ntx_loss * 0.5 + 0.5 * criterion_bce(outputs.squeeze(-1), batch_patient_labels)
                    loss = loss.mean()
                    loss += -mlls(gp_outputs, batch_patient_labels) * 0.5
                else:
                    # loss = -mlls(gp_outputs, batch_patient_labels) * 0.5 + 0.5 * criterion_bce(outputs.squeeze(-1), batch_patient_labels)
                    loss = -mlls(gp_outputs, batch_patient_labels)

                loss = loss.mean()
                preds = likelihoods(gp_outputs).probs >= THRESHOLD
                # preds = (torch.sigmoid(outputs) >= THRESHOLD).int()

            predictions.extend(preds.cpu().detach().numpy())

            total_loss += loss.item()
            loss.backward()
            optimizer.step()
            scheduler.step()
            if GP_MODEL == 'single_task':
                if NUM_CLASSES != 1:
                    for i in range(NUM_CLASSES):
                        variational_ngd_optimizer[i].step()
                else:
                    variational_ngd_optimizer.step()

        if NUM_CLASSES == 1:
            labels.extend(batch_patient_labels.cpu().numpy())
        else:
            labels.extend(batch_multi_labels.cpu().numpy())

    return total_loss / len(data_loader), predictions, labels


def validate(model, likelihoods, data_loader, criterion_cl, criterion_bce, mlls, device):
    """Validate the model."""
    model.eval()
    likelihoods.eval()

    total_loss = 0.0
    alpha = 0.5
    predictions = []
    labels = []

    with (torch.inference_mode()):
        for batch_data, batch_labels, batch_patient_labels, batch_multi_labels in tqdm(data_loader):
            batch_data = batch_data.to(device)
            batch_patient_labels = batch_patient_labels.float().to(device)
            batch_multi_labels = batch_multi_labels.float().to(device)

            if TRAINING_TYPE == 'end_to_end':
                outputs, gp_outputs, att_outputs = model(batch_data)
                if GP_MODEL == 'single_task':
                    loss = 0
                    if NUM_CLASSES != 1:
                        for i in range(NUM_CLASSES):
                            loss += -mlls[i](gp_outputs[i], batch_multi_labels[:, i])

                        loss.mean()
                        loss += 0.5 * criterion_bce(outputs, batch_multi_labels)
                        total_loss += loss.item()

                        probabilities = torch.stack([likelihoods[i](gp_outputs[i]).probs for i in range(NUM_CLASSES)],
                                                    dim=1)
                        preds = (probabilities >= THRESHOLD).int()
                    else:
                        loss = -mlls(gp_outputs, batch_patient_labels) * 0.5 + 0.5 * criterion_bce(outputs.squeeze(-1),
                                                                                                   batch_patient_labels)
                        # loss = -mlls(gp_outputs, batch_patient_labels)
                        loss = loss.mean()
                        total_loss += loss.item()
                        preds = likelihoods(gp_outputs).probs >= THRESHOLD
                        # preds = (torch.sigmoid(outputs) >= THRESHOLD).int()

                predictions.extend(preds.cpu().detach().numpy())

            if NUM_CLASSES == 1:
                labels.extend(batch_patient_labels.cpu().numpy())
            else:
                labels.extend(batch_multi_labels.cpu().numpy())
    return total_loss / len(data_loader), predictions, labels


def train_model(model, likelihoods, train_loader, val_loader, criterion_cl, criterion_bce, optimizer, num_epochs,
                learning_rate, device='cuda'):
    """Train the model and return the best model based on validation accuracy."""
    model = model.to(device)
    model.train()
    likelihoods.train()

    # Initialize Early Stopping
    early_stopping = EarlyStopping(patience=10, verbose=True)

    if GP_MODEL == 'single_task':
        if NUM_CLASSES != 1:
            mlls = [
                gpytorch.mlls.VariationalELBO(likelihoods[i], model.gp_layers[i], num_data=len(train_loader.dataset))
                for
                i in range(NUM_CLASSES)]
            mlls = [mll.to(device) for mll in mlls]

            variational_ngd_optimizer = [
                gpytorch.optim.NGD(model.gp_layers[i].variational_parameters(), num_data=len(train_loader.dataset),
                                   lr=LEARNING_RATE_NGD) for i in range(NUM_CLASSES)]
        else:
            mlls = gpytorch.mlls.VariationalELBO(likelihoods, model.gp_layers, num_data=len(train_loader.dataset))
            mlls = mlls.to(device)
            variational_ngd_optimizer = gpytorch.optim.NGD(model.gp_layers.variational_parameters(),
                                                           num_data=len(train_loader.dataset),
                                                           lr=LEARNING_RATE_NGD)

    # scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=learning_rate, steps_per_epoch=len(train_loader), epochs=num_epochs)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=5, eta_min=1e-5)

    best_val_accuracy = 0.0
    best_model_state = None
    best_likelihood_state = None

    for epoch in range(num_epochs):
        train_loss, train_predictions, train_labels = train_epoch(model, likelihoods, train_loader, criterion_cl,
                                                                  criterion_bce,
                                                                  mlls, optimizer, variational_ngd_optimizer,
                                                                  scheduler, device)
        train_metrics = calculate_metrics(train_predictions, train_labels)
        print_epoch_stats(epoch, num_epochs, "train", train_loss, train_metrics)

        wandb.log({
            "train/loss": train_loss,
            "train/accuracy": train_metrics["accuracy"],
            "train/precision": train_metrics["precision"],
            "train/recall": train_metrics["recall"],
            "train/f1": train_metrics["f1"],
            "train/auc": train_metrics["auc"],
            "train/cohen_kappa": train_metrics["cohen_kappa"]
        })

        # Validation phase
        val_loss, val_predictions, val_labels = validate(model, likelihoods, val_loader, criterion_cl, criterion_bce,
                                                         mlls,
                                                         device)
        val_metrics = calculate_metrics(val_predictions, val_labels)
        print_epoch_stats(epoch, num_epochs, "validation", val_loss, val_metrics)

        wandb.log({
            "val/loss": val_loss,
            "val/accuracy": val_metrics["accuracy"],
            "val/precision": val_metrics["precision"],
            "val/recall": val_metrics["recall"],
            "val/f1": val_metrics["f1"],
            "val/auc": val_metrics["auc"],
            "val/cohen_kappa": val_metrics["cohen_kappa"]
        })

        # Early Stopping Check
        early_stopping(val_metrics["auc"], model)
        if early_stopping.early_stop:
            print("Early stopping triggered.")
            break

        # Save best model
        if val_metrics['auc'] > best_val_accuracy:
            best_val_accuracy = val_metrics['auc']
            best_model_state = model.state_dict()
            best_likelihood_state = likelihoods.state_dict()

    # Load best model
    if best_model_state and best_likelihood_state:
        model.load_state_dict(best_model_state)
        likelihoods.load_state_dict(best_likelihood_state)
    # Optionally log the best model to W&B (if desired)
    print(f'Best Validation AUC: {best_val_accuracy}')
    wandb.log_artifact(wandb.Artifact("best_model", type="model", metadata={"accuracy": best_val_accuracy}))

    return model, likelihoods

## Model Evaluation

In [24]:
## Model Evaluation Functions
def evaluate_model(model, likelihoods, data_loader, device='cuda'):
    """Evaluate the model on the given data loader."""
    model = model.to(device)
    likelihoods = likelihoods.to(device)
    model.eval()
    likelihoods.eval()

    predictions = []
    labels = []
    probs = []

    with torch.inference_mode():
        for batch_data, batch_labels, batch_patient_labels, batch_multi_labels in data_loader:
            batch_data = batch_data.to(device)
            batch_patient_labels = batch_patient_labels.float().to(device)
            batch_multi_labels = batch_multi_labels.float().to(device)

            if TRAINING_TYPE == 'end_to_end':
                outputs, gp_outputs, att_outputs = model(batch_data)
                if GP_MODEL == 'single_task':
                    if NUM_CLASSES > 1:
                        probabilities = torch.stack([likelihoods[i](gp_outputs[i]).probs for i in range(NUM_CLASSES)],
                                                    dim=1)
                    else:
                        probabilities = likelihoods(gp_outputs).probs
                    preds = (probabilities >= THRESHOLD).int()
                    # preds = (torch.sigmoid(outputs) >= THRESHOLD).int()
                predictions.extend(preds.cpu().detach().numpy())

            if NUM_CLASSES == 1:
                labels.extend(batch_patient_labels.cpu().numpy())
            else:
                labels.extend(batch_multi_labels.cpu().numpy())

            probs.extend(probabilities.cpu().detach().numpy())

    return np.array(predictions), np.array(labels), np.array(probs)


def print_metrics(metrics):
    """Print the calculated metrics."""
    print(f"Test Accuracy: {metrics['accuracy']:.4f}, "
          f"Precision: {metrics['precision']:.4f}, "
          f"Recall: {metrics['recall']:.4f}, "
          f"F1: {metrics['f1']:.4f}",
          f"AUC: {metrics['auc']:.4f}",
          f"Cohen Kappa: {metrics['cohen_kappa']:.4f}")

# Visualization

In [25]:
## Visualization Functions
def plot_roc_curve(model, likelihoods, data_loader, device):
    """Plot the ROC curve for the model predictions."""
    model.eval()
    if likelihoods:
        likelihoods.eval()
    labels = []
    predictions = []

    with torch.no_grad():
        for batch_data, batch_labels, batch_patient_labels, batch_multi_labels in data_loader:
            batch_data = batch_data.to(device)
            batch_patient_labels = batch_patient_labels.float().to(device)
            batch_multi_labels = batch_multi_labels.float().to(device)

            if TRAINING_TYPE == 'end_to_end':
                outputs, gp_outputs, _ = model(batch_data)
                if GP_MODEL == 'single_task':
                    if NUM_CLASSES > 1:
                        probabilities = torch.stack([likelihoods[i](gp_outputs[i]).probs for i in range(NUM_CLASSES)],
                                                    dim=1)
                    else:
                        probabilities = likelihoods(gp_outputs).probs
                    preds = probabilities

                else:
                    preds = torch.sigmoid(outputs)
                predictions.extend(preds.cpu().detach().numpy())
            else:
                outputs, cnn_features, attention_weights = model(batch_data)
                preds = torch.sigmoid(outputs)
                predictions.extend(preds.cpu().detach().numpy())

            if NUM_CLASSES == 1:  # Binary classification
                labels.extend(batch_patient_labels.cpu().numpy())
            else:
                labels.extend(batch_multi_labels.cpu().numpy())

    labels = np.array(labels)
    predictions = np.array(predictions)

    if NUM_CLASSES == 1:
        fpr, tpr, _ = roc_curve(labels, predictions)
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
    else:
        class_name = ['any', 'epidural', 'intraparenchymal', 'intraventricular', 'subarachnoid', 'subdural']
        for i in range(NUM_CLASSES):
            fpr, tpr, _ = roc_curve(labels[:, i], predictions[:, i])
            roc_auc = auc(fpr, tpr)
            plt.plot(fpr, tpr, lw=2,
                     label=f'Class {class_name[i]} (AUC = {roc_auc:.4f})')

    plt.plot([0, 1], [0, 1], 'k--', lw=2)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curves')
    plt.legend(loc="lower right")
    plt.show()

def plot_confusion_matrix(model, likelihoods, data_loader, criterion_cl, criterion_bce, device):
    """Plot confusion matrix for classification tasks."""
    if likelihoods:
        predictions, labels, _ = evaluate_model(model, likelihoods, data_loader, device)
    else:
        predictions, labels, _ = evaluate_phase_1(model, data_loader, criterion_cl, criterion_bce, device)
    if NUM_CLASSES > 1:  # Multi-label/multi-class
        cm = multilabel_confusion_matrix(labels, predictions)
        _, ax = plt.subplots(1, NUM_CLASSES, figsize=(15, 3))
        for i in range(NUM_CLASSES):
            ConfusionMatrixDisplay(cm[i]).plot(ax=ax[i])
            ax[i].set_title(f'Class {i}')
    else:  # Binary
        cm = confusion_matrix(labels, predictions)
        ConfusionMatrixDisplay(cm).plot()

    plt.tight_layout()
    plt.show()

# Model Helper Functions

In [26]:
## Data Processing Functions
def load_model(model_class, model_path, params):
    """Load a trained model from a file."""
    if not os.path.exists(model_path):
        raise FileNotFoundError(f"Model file not found at {model_path}")

    model = model_class(params)
    try:
        state_dict = torch.load(model_path, map_location=torch.device('cuda'), weights_only=True)
        if not state_dict:
            raise ValueError(f"The state dictionary loaded from {model_path} is empty")
        model.load_state_dict(state_dict)
    except Exception as e:
        print(f"Error loading model from {model_path}: {str(e)}")
        print("Initializing model with random weights instead.")
        return model  # Return the model with random initialization

    return model.eval()


def get_test_results(model, test_loader, test_labels, device=DEVICE):
    """Get test results including patient information."""
    predictions, _ = evaluate_model(model, test_loader, device)

    results = []
    for i, row in enumerate(test_labels.itertuples(index=False)):
        result = {col: getattr(row, col) for col in test_labels.columns}
        result['prediction'] = predictions[i]
        results.append(result)

    return pd.DataFrame(results)

## Model Weight Initialization

In [27]:
# If you need to use the Glorot (Xavier) uniform initialization
def init_weights(m):
    if type(m) == nn.Conv2d:
        torch.nn.init.xavier_uniform_(m.weight)

# Main

In [28]:

def main(mode='train', use_cv=False, num_folds=5):
    # os.environ["WANDB_DISABLED"] = "true"
    current_time = datetime.now().strftime("%Y%m%d_%H%M")
    run_name = f"{TRAINING_TYPE}_experiment_{current_time}_{GP_MODEL}_refiner_fc_{PROJECTION_HIDDEN_DIM}_output_{PROJECTION_OUTPUT_DIM}_attention_{ATTENTION_HIDDEN_DIM}_kernel_{GP_KERNEL}_model_{MODEL_TYPE}"

    # Initialize W&B with a specific run name
    wandb.init(project="MIL_Resnet_ICH", name=run_name)

    # Log hyperparameters
    config = wandb.config
    config.learning_rate = LEARNING_RATE
    config.batch_size = TRAIN_BATCH_SIZE
    config.num_epochs = NUM_EPOCHS

    params = {
        'channels': CHANNELS,  # Number of input channels (e.g., 1 for grayscale, 3 for RGB)
        'num_classes': NUM_CLASSES,  # Number of output classes for classification
        'drop_prob': 0.5,  # Dropout probability
        'inducing_points': INDUCING_POINTS,  # Number of inducing points for the Gaussian Process layer
        'projection_location': PROJECTION_LOCATION,  # Choose from 'after_resnet', 'after_attention', or 'after_gp'
        'projection_hidden_dim': PROJECTION_HIDDEN_DIM,  # Hidden dimension size for the projection head
        'projection_output_dim': PROJECTION_OUTPUT_DIM,  # Output dimension size for the projection head
        'attention_hidden_dim': ATTENTION_HIDDEN_DIM,  # Hidden dimension size for the attention head
        'gp_model': GP_MODEL,
        'kernel_type': GP_KERNEL,
        'model_type': MODEL_TYPE,
        'contrastive_learning': CONTRASTIVE_LEARNING
    }

    if use_cv == False:
        if NUM_CLASSES == 1:
            train_labels, val_labels, test_labels = split_dataset(patient_scan_labels, test_size=0.0)
        else:
            train_labels, val_labels, test_labels = split_dataset_for_multilabel(patient_scan_labels, test_size=0.0)
            
        test_dir = './data_analyze/testing_dataset_150_redundancy.csv' if DATA_REDUNDANCY else \
                    './data_analyze/testing_dataset_150.csv'
        test_labels = pd.read_csv(test_dir)

        train_loader = get_train_loader(dicom_dir, train_labels, batch_size=TRAIN_BATCH_SIZE)
        val_loader = get_train_loader(dicom_dir, val_labels, batch_size=VALID_BATCH_SIZE)
        test_loader = get_test_loader(dicom_dir, test_labels, batch_size=TEST_BATCH_SIZE)

        if TRAINING_TYPE == 'end_to_end':
            if GP_MODEL == 'single_task':
                model = MILModels.CNN_ATT_GP_Multilabel(params)

                if NUM_CLASSES != 1:
                    likelihood = nn.ModuleList([PGLikelihood() for _ in range(NUM_CLASSES)])
                else:
                    likelihood = PGLikelihood()
                    # likelihood = gpytorch.likelihoods.BernoulliLikelihood()
                optimizer = optim.Adam([
                    {'params': model.parameters(), 'lr': config.learning_rate},
                    {'params': likelihood.parameters(), 'lr': config.learning_rate}
                ])
        elif TRAINING_TYPE == 'cnn_att':
            model = MILModels.CNN_Attention(params)
            likelihood = None
            optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)

        criterion_cl = NTXentLoss(0.5)
        if NUM_CLASSES == 1:
            pos_weights = torch.tensor([0.5]).to(DEVICE)
        else:
            pos_weights = torch.tensor([0.5] * NUM_CLASSES).to(DEVICE)
        criterion_bce = nn.BCEWithLogitsLoss(pos_weight=pos_weights)

        if mode == 'train':
            wandb.watch(model)  # Watch the model to log gradients and parameters
            if TRAINING_TYPE == 'end_to_end':
                # model.apply(init_weights)
                trained_model, likelihood = train_model(model, likelihood, train_loader, val_loader, criterion_cl,
                                                        criterion_bce, optimizer, config.num_epochs, config.learning_rate, DEVICE)
                predictions, labels, _ = evaluate_model(trained_model, likelihood, test_loader, DEVICE)
            else:
                print("Training Phase 1: CNN + ATT")
                trained_model, _ = train_model_phase_1(model, train_loader, val_loader, criterion_cl, criterion_bce,
                                                        optimizer, config.num_epochs, DEVICE)
                predictions, labels, _ = evaluate_phase_1(trained_model, test_loader, criterion_cl, criterion_bce, DEVICE)

            metrics = calculate_metrics(predictions, labels, average='binary')
            wandb.log(metrics)
            print_metrics(metrics)

            plot_roc_curve(trained_model, likelihood, test_loader, DEVICE)
            plot_confusion_matrix(trained_model, likelihood, test_loader, criterion_cl, criterion_bce, DEVICE)
            torch.save(trained_model.state_dict(), MODEL_PATH)
            if likelihood:
                torch.save(likelihood.state_dict(), LIKELIHOOD_PATH)
        else:
            if TRAINING_TYPE == 'end_to_end':
                if GP_MODEL == 'single_task':
                    trained_model = load_model(MILModels.CNN_ATT_GP_Multilabel, MODEL_PATH, params)
                    predictions, labels, probs = evaluate_model(trained_model, likelihood, test_loader, DEVICE)
            else:
                trained_model = load_model(MILModels.CNN_Attention, MODEL_PATH, params)
                predictions, labels, probs = evaluate_phase_1(trained_model, test_loader, criterion_cl, criterion_bce, DEVICE)

            metrics = calculate_metrics(predictions, labels)
            wandb.log(metrics)
            print_metrics(metrics)

            plot_roc_curve(trained_model, likelihood, test_loader, DEVICE)
            plot_confusion_matrix(trained_model, likelihood, test_loader, criterion_cl, criterion_bce, DEVICE)

            print("\nProbabilities for each row in the test set:")
            for i, prob in enumerate(probs):
                if NUM_CLASSES == 1:
                    is_correct = predictions[i] == labels[i]
                    print(
                        f"Row {i + 1}: Probs: {prob.item():.4f} | Prediction: {predictions[i]} | Label: {labels[i]} | Result: {'True' if is_correct else 'False'}")
                else:
                    is_correct = np.array_equal(predictions[i], labels[i])
                    print(
                        f"Row {i + 1}: Probs: {prob.tolist()} | Prediction: {predictions[i].tolist()} | Label: {labels[i].tolist()} | Result: {'True' if is_correct else 'False'}")

    else:
        if NUM_CLASSES == 1:
            trainval_labels, test_labels, _ = split_dataset(
                patient_scan_labels,
                val_size=TEST_SIZE,
                test_size=0.0
            )
            test_labels = pd.read_csv('./data_analyze/testing_dataset_150_redundancy.csv')
        else:
            trainval_labels, test_labels, _ = split_dataset_for_multilabel(
                patient_scan_labels,
                val_size=TEST_SIZE,
                test_size=0.0
            )

        # Setup CV using trainval data
        kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
        fold_metrics = []

        for fold_idx, (train_idx, val_idx) in enumerate(kf.split(trainval_labels)):
            print(f"\nFold {fold_idx + 1}/{num_folds}")

            # Create fold splits
            train_labels_fold = trainval_labels.iloc[train_idx]
            val_labels_fold = trainval_labels.iloc[val_idx]

            # Create data loaders
            train_loader = get_train_loader(dicom_dir, train_labels_fold, TRAIN_BATCH_SIZE)
            val_loader = get_train_loader(dicom_dir, val_labels_fold, VALID_BATCH_SIZE)
            test_loader = get_test_loader(dicom_dir, test_labels, TEST_BATCH_SIZE)

            # Initialize model
            if GP_MODEL == 'single_task':
                model = MILModels.CNN_ATT_GP_Multilabel(params)
                if NUM_CLASSES != 1:
                    likelihood = nn.ModuleList([PGLikelihood() for _ in range(NUM_CLASSES)])
                else:
                    likelihood = PGLikelihood()
                optimizer = optim.Adam([
                    {'params': model.parameters(), 'lr': config.learning_rate},
                    {'params': likelihood.parameters(), 'lr': config.learning_rate}
                ])


            if NUM_CLASSES == 1:
                pos_weights = torch.tensor([5.0]).to(DEVICE)
            else:
                pos_weights = torch.tensor([5.0] * NUM_CLASSES).to(DEVICE)
            criterion_bce = nn.BCEWithLogitsLoss(pos_weight=pos_weights) # Weighted BCE Loss
            criterion_cl = NTXentLoss(0.5) # Contrastive Learning Loss

            # Train model
            wandb.init(project="MIL_Resnet_ICH", name=f"{run_name}_fold_{fold_idx + 1}")
            wandb.watch(model)
            trained_model, likelihood = train_model(model, likelihood, train_loader, val_loader, criterion_cl,
                                                    criterion_bce,
                                                    optimizer, config.num_epochs, config.learning_rate, DEVICE)

            # Evaluate model
            if TRAINING_TYPE == 'end_to_end':
                predictions, labels, _ = evaluate_model(trained_model, likelihood, test_loader, DEVICE)
            else:
                predictions, labels, _ = evaluate_phase_1(trained_model, test_loader, criterion_cl, criterion_bce, DEVICE)
            metrics = calculate_metrics(predictions, labels)

            # Log metrics
            wandb.log(metrics)
            print_metrics(metrics)
            fold_metrics.append(metrics)

            plot_roc_curve(trained_model, likelihood, test_loader, DEVICE)
            plot_confusion_matrix(trained_model, likelihood, test_loader, DEVICE)

            # Save model
            torch.save(trained_model.state_dict(), f"{MODEL_PATH}_fold_{fold_idx + 1}")
            if likelihood:
                torch.save(likelihood.state_dict(), f"{LIKELIHOOD_PATH}_fold_{fold_idx + 1}")

        # Calculate average metrics across all folds
        avg_metrics = {}
        for metric in fold_metrics[0].keys():
            avg_metrics[metric] = np.mean([fold[metric] for fold in fold_metrics])
        print("\nAverage Metrics Across All Folds:")
        print_metrics(avg_metrics)
        wandb.log(avg_metrics)

# Results

In [ ]:
if __name__ == "__main__":
    main(mode='train', use_cv=False, num_folds=5)

In [30]:
# import torch
# import torch.nn as nn
# import torchsummary
#
# class VGG(nn.Module):
#     def __init__(self, input_channels=3):
#         super(VGG, self).__init__()
#
#         self.features = nn.Sequential(
#             # Conv1
#             nn.Conv2d(input_channels, 16, kernel_size=5, padding=2),
#             nn.ReLU(),
#             nn.BatchNorm2d(16),
#             nn.MaxPool2d(kernel_size=2, stride=2),
#
#             # Conv2
#             nn.Conv2d(16, 32, kernel_size=3, padding=0),
#             nn.ReLU(),
#             nn.BatchNorm2d(32),
#             nn.MaxPool2d(kernel_size=2, stride=2),
#             nn.Dropout(0.3),
#
#             # Conv3
#             nn.Conv2d(32, 32, kernel_size=3, padding=0),
#             nn.ReLU(),
#             nn.BatchNorm2d(32),
#             nn.MaxPool2d(kernel_size=2, stride=2),
#
#             # Conv4
#             nn.Conv2d(32, 32, kernel_size=3, padding=2),
#             nn.ReLU(),
#             nn.BatchNorm2d(32),
#             nn.MaxPool2d(kernel_size=2, stride=2),
#
#             # Conv5
#             nn.Conv2d(32, 32, kernel_size=3, padding=0),
#             nn.ReLU(),
#             nn.BatchNorm2d(32),
#             nn.MaxPool2d(kernel_size=2, stride=2),
#             nn.Dropout(0.3),
#
#             # Conv6
#             nn.Conv2d(32, 32, kernel_size=3, padding=0),
#             nn.ReLU(),
#             nn.BatchNorm2d(32),
#             nn.MaxPool2d(kernel_size=2, stride=2),
#             nn.Dropout(0.3)
#         )
#
#         self.flatten = nn.Flatten()
#
#     def forward(self, x):
#         x = self.features(x)
#         x = self.flatten(x)
#         return x
#
# # Instantiate the model
# model = VGG(input_channels=1)
#
# # If you need to use the Glorot (Xavier) uniform initialization
# def init_weights(m):
#     if type(m) == nn.Conv2d:
#         torch.nn.init.xavier_uniform_(m.weight)
#
# model.apply(init_weights)
#
# # # Test the model with a sample input
# # input_tensor = torch.randn(1, 1, 512, 512)
# # output = model(input_tensor)
# # print(f"Output shape: {output.shape}")
#
# # # Print the model architecture
# model_info = torchsummary.summary(model, (1, 512, 512), device='cpu')
# print(model_info)